<a href="https://colab.research.google.com/github/sj0054pe/MBTI_Estim/blob/master/src/5_BERT_%E6%96%87%E6%9B%B8%E5%88%86%E9%A1%9E.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#<center>BERT Twitter 文書分類 </center>

###<center>参考サイト</center>

###<center>https://qiita.com/takubb/items/fd972f0ac3dba909c293</center>

1. 準備（ライブラリのインストール）
2. データセットの準備
3. データの前処理（BERTが受けとれる形式にデータを整形）
4. 学習済みモデルのロード
5. 訓練（Pre-trainedモデルのファインチューニング）
6. 検証

#0. 作業環境の準備

##0.1. google driveのマウント

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


##0.2. カレントディレクトリへ移動

In [ ]:
%cd /content/drive/MyDrive/研究室/研究_M2/src

/content/drive/.shortcut-targets-by-id/1OWWQmFsr-0HoCVHJBaVtdmaZ4m43kkOk/研究_M2/src


##0.3. GPUへ接続

In [ ]:
import torch
# GPUが使えれば利用する設定
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

##0.4. 現在時刻の取得

In [ ]:
import datetime
import pytz

def pull_Datetime():
  DATETIME_now = datetime.datetime.now(pytz.timezone('Asia/Tokyo'))
  #print(DATETIME_now)

  return DATETIME_now

##0.5. 分類モデルの性格指標を指定

この指標をもとに、2クラス分類を行う。

E or I なら
EI指標の分類器を作成する。

In [ ]:
TYPE_INDEX='EorI_E'
#TYPE_INDEX='NorS_N'
#TYPE_INDEX='ForT_F'
#TYPE_INDEX='JorP_J'

##0.6. 学習するデータセットの指定

ツイッターの各投稿に、MBTIの16性格がラベリングされている。


In [ ]:
INPUT_CSV_v5='./Assets/Assets_Output/Tweet_Datasets/v5_forBERT.csv'

#1. 準備 (ライブラリのインストール)

Hugging Face Library 'Transformer' のダウンロード

* transformers
* 最新のNLPモデルを利用できるライブラリ
* BERTの他にもGPT-2, RoBERTa, XLM, DistilBert, XLNet, T5, CTRL...などの最新モデルや100言語以上の学習済みモデル（Pre-trained model）を利用可能

In [ ]:
!pip install transformers

!apt install aptitude swig
!aptitude install mecab libmecab-dev mecab-ipadic-utf8 git make curl xz-utils file -y
!pip install mecab-python3

#参考サイト (https://qiita.com/takubb/items/fd972f0ac3dba909c293) では欠けているライブラリ
!pip install fugashi
!pip install ipadic

     |████████████████████████████████| 2.8 MB 5.3 MB/s 
     |████████████████████████████████| 3.3 MB 51.3 MB/s 
     |████████████████████████████████| 52 kB 1.8 MB/s 
     |████████████████████████████████| 895 kB 48.0 MB/s 
     |████████████████████████████████| 636 kB 40.5 MB/s 
  Attempting uninstall: pyyaml
    Found existing installation: PyYAML 3.13
    Uninstalling PyYAML-3.13:
      Successfully uninstalled PyYAML-3.13
Reading package lists... Done
Building dependency tree       
Reading state information... Done
The following additional packages will be installed:
  aptitude-common libcgi-fast-perl libcgi-pm-perl libclass-accessor-perl
  libcwidget3v5 libencode-locale-perl libfcgi-perl libhtml-parser-perl
  libhtml-tagset-perl libhttp-date-perl libhttp-message-perl libio-html-perl
  libio-string-perl liblwp-mediatypes-perl libparse-debianchangelog-perl
  libsigc++-2.0-0v5 libsub-name-perl libtimedate-perl liburi-perl libxapian30
  swig3.0
Suggested packages:
  aptitude-do

#2. データセットの準備

##2.1.準備したツイートデータをダウンロードする。

In [ ]:
import pandas as pd
df=pd.read_csv('%s' % INPUT_CSV_v5, engine='python')

# データの確認
print(f'データサイズ： {df.shape}')

df.sample(10)



---



##2.2. 必要なデータ（文章とラベル）だけ抽出します

In [ ]:
df.drop(['notes', 'Unnamed: 0', 'Unnamed: 0.1', 'Unnamed: 0.1.1',  'Unnamed: 0.1.1.1'],axis=1, inplace=True)
df.dropna(how='any', axis=0, inplace=True)

In [ ]:
df=pd.get_dummies(df, columns=["label_EorI", "label_NorS", "label_ForT", "label_JorP"], sparse=True)
df

##2.3. 文章データをsentences、ラベルデータを labelsという変数に保存します

次パートからはこの2変数だけを利用します

In [ ]:
import random
 #データの抽出
df=df.sample(n=10000)

sentences= df['text'].values #sentences = df.sentence.values
labels= df['label_%s' % TYPE_INDEX].values #labels = df.label.values

In [ ]:
labels 

[0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 1, 1, 0, 0, 0, 0, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 1, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 1, 0, 0, 1, 0, 0, 0, 0, 0, 1, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, ...]
Fill: 0
IntIndex
Indices: array([   2,   11,   13, ..., 9986, 9992, 9999], dtype=int32)

In [ ]:
print(len(sentences))

10000


#3. データの前処理（BERTが受けとれる形式にデータを整形）

BERTでは基本的に学習済みモデルを利用する為、そのモデルが読み込めるフォーマットにデータを変換する必要があります
具体的には、以下の4つ手続きが必要になります

1. BERT Tokenizerを用いて単語分割・IDへ変換
>* 学習済みモデルの作成時と同じtokenizer（形態素解析器）を利用する必要がある
>* 日本語ではMecabやJuman++を利用されることが多い

2. Special tokenの追加
>* 文章の最後に[SEP]という単語する
>* 文章のはじめに[CLS]という単語を追加する（分類問題に利用される）
3. 文章の長さの固定
>* BERTでは全ての文書の長さ（単語の数）を同じにする必要がある（1文章あたりの最大の単語数は512単語）
>* そこで、Padding/Truncatingを用いて固定長に変換する
>* Paddingとは、指定した長さに満たない文章を[Pad]という意味を持たない単語の埋める処理
>* Truncatingとは、指定した長さを超える単語を切り捨てること
4. Attention mask arrayの作成
>* [Padding]を0、それ以外のTokenを1とした配列
>* 一見ややこししそう見えますが、どれもTransformerのライブラリでサポートされているので、簡単に実行できます

また、以下のイメージも参考にモデルの全体像と照らし合わせながら前処理内容をみるとイメージが沸きやすいかもしれません

##3.1. BERT Tokenizerを用いて単語分割・IDへ変換

In [ ]:
## Tokenizerの準備
from transformers import BertJapaneseTokenizer
tokenizer = BertJapaneseTokenizer.from_pretrained('cl-tohoku/bert-base-japanese-whole-word-masking')

#念の為、確認
## テスト実行
# 元文章
print(' Original: ', sentences[0])
# Tokenizer
print('Tokenized: ', tokenizer.tokenize(sentences[0]))
# Token-id
print('Token IDs: ', tokenizer.convert_tokens_to_ids(tokenizer.tokenize(sentences[0])))

Downloading:   0%|          | 0.00/258k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/110 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/479 [00:00<?, ?B/s]

 Original:  「その嘘のために真実さえ壊すと決めた」
アプリ「テイルズ オブ ザ レイズ ミラージュ プリズン」2018年配信決定！
事前登録で豪華アイテムをゲット！
https://t.co/dhb1TKGoHT
#ザレイズ
Tokenized:  ['「', 'その', '嘘', 'の', 'ため', 'に', '真実', 'さえ', '壊', '##す', 'と', '決め', 'た', '」', 'アプリ', '「', 'テイ', '##ルズ', 'オブ', 'ザ', 'レイ', '##ズ', 'ミラージュ', 'プリ', '##ズ', '##ン', '」', '2018', '年', '配信', '決定', '!', '事前', '登録', 'で', '豪華', 'アイテム', 'を', 'ゲッ', '##ト', '!', 'http', '##s', '://', 't', '.', 'co', '/', 'd', '##h', '##b', '1', 'T', '##K', '##G', '##o', '##HT', '#', 'ザ', '##レイズ']
Token IDs:  [36, 59, 11753, 5, 82, 7, 10678, 6155, 4945, 28484, 13, 2372, 10, 38, 4437, 36, 15908, 4892, 4123, 1363, 2995, 28582, 28127, 3898, 28582, 28459, 38, 1156, 19, 2393, 1067, 679, 6325, 1800, 12, 11934, 5405, 11, 18149, 28476, 679, 21313, 28589, 16831, 822, 143, 20852, 465, 1267, 28695, 28896, 17, 260, 28847, 28770, 28538, 12993, 4931, 1363, 20992]


##3.2. 次に文章の長さを固定する為に、文章あたりの最大単語数を確認します

ここでは、以下の方法で最大単語数を確認していますが、最大単語数は決め打ちで設定することも可能です

In [ ]:
# print(sentences[15806])

In [ ]:
from tqdm.notebook import tqdm
import numpy as np

# 最大単語数の確認
max_len = []
# 1文づつ処理
len_sentences = len(sentences)
for sent in tqdm(sentences):
    # Tokenizeで分割
    token_words = tokenizer.tokenize(sent)
    # 文章数を取得してリストへ格納
    max_len.append(len(token_words))
# 最大の値を確認
len_max=max(max_len)
#len_q0, len_q25, len_q50, len_q75, len_q100=np.percentile(max_len,  [0, 25, 50, 75, 100])
#len_q75=int(len_q75)
print('最大単語数: ', max(max_len))
print('上記の最大単語数にSpecial token（[CLS], [SEP]）の+2をした値が最大単語数')

  0%|          | 0/10000 [00:00<?, ?it/s]

最大単語数:  133
上記の最大単語数にSpecial token（[CLS], [SEP]）の+2をした値が最大単語数


##3.3. Tokenizerと最大単語数の確認がとれたので、全ての文章に一括して処理を行います

tokenizer.encode_plusを利用するとまとめて処理が行えます

In [ ]:
from tqdm.notebook import tqdm

input_ids = []
attention_masks = []

# 1文づつ処理
for sent in tqdm(sentences):
    encoded_dict = tokenizer.encode_plus(
                        sent,                      
                        add_special_tokens = True, # Special Tokenの追加
                        max_length = len_max,        # 文章の長さを固定（Padding/Trancatinating）#mayo
                        pad_to_max_length = True,# PADDINGで埋める
                        return_attention_mask = True,   # Attention maksの作成
                        return_tensors = 'pt',     #  Pytorch tensorsで返す
                   )

    # 単語IDを取得    
    input_ids.append(encoded_dict['input_ids'])

    # Attention　maskの取得
    attention_masks.append(encoded_dict['attention_mask'])

# リストに入ったtensorを縦方向（dim=0）へ結合
input_ids = torch.cat(input_ids, dim=0)
attention_masks = torch.cat(attention_masks, dim=0)

  0%|          | 0/10000 [00:00<?, ?it/s]

Truncation was not explicitly activated but `max_length` is provided a specific value, please use `truncation=True` to explicitly truncate examples to max length. Defaulting to 'longest_first' truncation strategy. If you encode pairs of sequences (GLUE-style) with the tokenizer you can select this strategy more precisely by providing a specific strategy to `truncation`.
/usr/local/lib/python3.7/dist-packages/transformers/tokenization_utils_base.py:2204: FutureWarning: The `pad_to_max_length` argument is deprecated and will be removed in a future version, use `padding=True` or `padding='longest'` to pad to the longest sequence in the batch, or use `padding='max_length'` to pad to a max length. In this case, you can give a specific length with `max_length` (e.g. `max_length=45`) or leave max_length to None to pad to the maximal input size of the model (e.g. 512 for Bert).
  FutureWarning,


##labelをtensor型に変換

labelはpd.get_dummiesでone-hotにしてます

In [ ]:
# tenosor型に変換
labels = torch.tensor(labels)

## BERTの前処理が正常に実行されたか確認

前処理は、

Special tokenの追加、文章の長さの固定、Attention mask arrayの作成

である。

In [ ]:
# 確認
print('Original: ', sentences[0])
print('Token IDs:', input_ids[0])

Original:  「その嘘のために真実さえ壊すと決めた」
アプリ「テイルズ オブ ザ レイズ ミラージュ プリズン」2018年配信決定！
事前登録で豪華アイテムをゲット！
https://t.co/dhb1TKGoHT
#ザレイズ
Token IDs: tensor([    2,    36,    59, 11753,     5,    82,     7, 10678,  6155,  4945,
        28484,    13,  2372,    10,    38,  4437,    36, 15908,  4892,  4123,
         1363,  2995, 28582, 28127,  3898, 28582, 28459,    38,  1156,    19,
         2393,  1067,   679,  6325,  1800,    12, 11934,  5405,    11, 18149,
        28476,   679, 21313, 28589, 16831,   822,   143, 20852,   465,  1267,
        28695, 28896,    17,   260, 28847, 28770, 28538, 12993,  4931,  1363,
        20992,     3,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
            0

1つ目の文章の前処理結果です

元々、日本語テキストだった文章がID化されています

Token IDsの最初に単語IDはSpecial tokenの[CLS]を表し、後半の0埋めが[Pad]を示しています



```
Original:  旧式Macで禁断のパワーアップ！最新PCやソフトを一挙にチェック
Token IDs: tensor([    2, 18718,  8653,    12,  1763, 29135,     5, 20734,   679,  6215,
         3794,    49,  1604,    11, 24598,     7,  9398,     3,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0])
```



##3.5. データローダの適用とデータの分割

ここまでで、前処理が完了したので、90%を訓練データ、10％をテストデータに分割して、pytorchで学習を行う為に、データローダーへ変換しておきます

データローダーの説明は、pytorchの基本的な操作なので本記事では割愛します（簡単に言うと、データをバッチごとに分割して、学習を上手くやってくれるデータ型です）

In [ ]:
from torch.utils.data import TensorDataset, random_split
from torch.utils.data import DataLoader, RandomSampler, SequentialSampler

# データセットクラスの作成
dataset = TensorDataset(input_ids, attention_masks, labels)

# 90%地点のIDを取得
# train_size = int(0.9 * len(dataset))
# val_size = len(dataset) - train_size
#train_dataset, val_dataset = random_split(dataset, [train_size, val_size])

# データセットを分割
from sklearn.model_selection import train_test_split
train_dataset, val_dataset  = train_test_split(dataset, train_size=0.8)
val_dataset, test_dataset  = train_test_split(val_dataset, train_size=0.8)

print('訓練データ数：{}'.format(len(train_dataset)))
print('検証データ数:　{} '.format(len(val_dataset )))
print('テストデータ数:　{} '.format(len(test_dataset)))

# データローダーの作成
batch_size = 64

# 訓練データローダー
train_dataloader = DataLoader(
            train_dataset,  
            sampler = RandomSampler(train_dataset), # ランダムにデータを取得してバッチ化
            batch_size = batch_size
        )

# 検証データローダー
validation_dataloader = DataLoader(
            val_dataset, 
            sampler = SequentialSampler(val_dataset), # 順番にデータを取得してバッチ化
            batch_size = batch_size
        )

# 検証データローダー
test_dataloader = DataLoader(
            test_dataset, 
            sampler = SequentialSampler(val_dataset), # 順番にデータを取得してバッチ化
            batch_size = batch_size
        )

訓練データ数：8000
検証データ数:　1600 
テストデータ数:　400 


#4. 学習済みモデルのロード

huggingface transformerのBERTファインチューニングでは以下のタスクをサポートしています
全てBERT Pre-trainedモデルをベースとして学習し、出力層のみそれぞれのタスクに適した構成になっています

* BertModel
* BertForPreTraining
*   BertForMaskedLM
*   BertForNextSentencePrediction
*   BertForSequenceClassification
*   BertForTokenClassification
* BertForQuestionAnswering

公式ドキュメント.

## BertForSequenceClassification

今回は2クラス分類問題なのでBertForSequenceClassificationを利用します

BertForSequenceClassificationは**「BERT学習済みモデルの最後の層に分類用のレイヤー追加したネットワーク構成」**です

データを入力することで、学習済みモデル全体と未学習部分の分類機レイヤーの学習が行われます

デフォルトでは、モデル全体のパラメーターがファインチューニングされますが、学習させるレイヤーの設定変更も可能です

モデルロード時にnum_labelsを変更することで多クラス分類問題へも対応できます

In [ ]:
from transformers import BertForSequenceClassification, AdamW, BertConfig

# BertForSequenceClassification 学習済みモデルのロード
model = BertForSequenceClassification.from_pretrained(
    "cl-tohoku/bert-base-japanese-whole-word-masking", # 日本語Pre trainedモデルの指定
    num_labels = 2, # ラベル数（今回はBinayなので2、数値を増やせばマルチラベルも対応可）
    output_attentions = False, # アテンションベクトルを出力するか
    output_hidden_states = False, # 隠れ層を出力するか
)

# モデルをGPUへ転送
model.cuda()

Downloading:   0%|          | 0.00/445M [00:00<?, ?B/s]

Some weights of the model checkpoint at cl-tohoku/bert-base-japanese-whole-word-masking were not used when initializing BertForSequenceClassification: ['cls.predictions.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.LayerNorm.weight', 'cls.seq_relationship.bias', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.decoder.weight', 'cls.predictions.transform.dense.bias', 'cls.seq_relationship.weight']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForSequenceClassification were not initialize

BertForSequenceClassification(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(32000, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0): BertLayer(
          (attention): BertAttention(
            (self): BertSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,), eps=1e-12, element

学習済みモデルのロードが完了しました

次にこのモデルを手元のデータに合うようにファインチューニングを行います

#5. 訓練（Pre-trainedモデルのファインチューニング）

モデルのファインチューニングは通常のニューラルネットワークの学習と同様で、損失関数と最適化手法（Optimizer）を指定して、学習ループ（任意のバッチサイズとエポックを指定）を回します

BERTでは損失関数はすでに定義されているので、ハイパーパラメーターとして、最適化手法とバッチサイズをエポック数だけ指定します

ここでは、論文（BERT paper, Appendix A.3）に従って、最適化関数はAdamW（Adam Weight Decay fix）を利用し、ハイパーパラメーターは下記のように設定します

Batch size: 32
Learning rate: 2e-5
Epochs: 4

In [ ]:
# 最適化手法の設定
optimizer = AdamW(model.parameters(), lr=2e-5)

# 訓練パートの定義
def train(model):
    model.train() # 訓練モードで実行
    train_loss = 0
    for batch in train_dataloader:# train_dataloaderはword_id, mask, labelを出力する点に注意
        b_input_ids = batch[0].to(device)
        b_input_mask = batch[1].to(device)
        b_labels = batch[2].to(device)
        optimizer.zero_grad()
        outputs = model(b_input_ids, 
                             token_type_ids=None, 
                             attention_mask=b_input_mask, 
                             labels=b_labels)
        loss = outputs.loss
        logits= outputs.logits
        loss.backward()
        torch.nn.utils.clip_grad_norm_(model.parameters(), 1.0)
        optimizer.step()
        train_loss += loss.item()
    return train_loss

# テストパートの定義
def validation(model):
    model.eval()# 訓練モードをオフ
    val_loss = 0
    with torch.no_grad(): # 勾配を計算しない
        for batch in validation_dataloader:
            b_input_ids = batch[0].to(device)
            b_input_mask = batch[1].to(device)
            b_labels = batch[2].to(device)
            with torch.no_grad():
                    outputs = model(b_input_ids, 
                                    token_type_ids=None, 
                                    attention_mask=b_input_mask,
                                    labels=b_labels)
            loss = outputs.loss
            logits= outputs.logits
            val_loss += loss.item()
    return val_loss

学習に必要な関数が定義できたので、学習を行う

In [ ]:
# 学習の実行
max_epoch = 10
train_loss_ = []
test_loss_ = []
import time
start = time.time()
epoch=0

# 学習途中の状態を読み込む。
# checkpoint = torch.load("./6_BERT_Huggingface/NS指標/model_NorS_N.tar")
# model.load_state_dict(checkpoint["model_state_dict"])
# optimizer.load_state_dict(checkpoint["optimizer_state_dict"])
# epoch = checkpoint["epoch"]

for epoch in range(epoch, max_epoch):
    print("%s epoch" % (epoch))
    train_ = train(model)

    # 学習途中の状態を保存する。 #https://pystyle.info/pytorch-save-and-load-model/
    torch.save(
        {
            "epoch": epoch,
            "model_state_dict": model.state_dict(),
            "optimizer_state_dict": optimizer.state_dict(),
         },
         "model_%s.tar" % TYPE_INDEX,
    )
  
    test_ = validation(model)
    train_loss_.append(train_)
    test_loss_.append(test_)
    
    elapsed_time = time.time() - start
    print("%s epoch  :  %s [sec] → %s" % (epoch, elapsed_time, pull_Datetime()))
    print()

print("終了時間 : %s [sec] → %s" % (elapsed_time, pull_Datetime()))

0 epoch
0 epoch  :  335.2268807888031 [sec] → 2021-09-26 21:15:47.951147+09:00

1 epoch
1 epoch  :  671.3648726940155 [sec] → 2021-09-26 21:21:24.052400+09:00

2 epoch
2 epoch  :  1007.3820405006409 [sec] → 2021-09-26 21:27:00.069579+09:00

3 epoch
3 epoch  :  1342.5309159755707 [sec] → 2021-09-26 21:32:35.218441+09:00

4 epoch
4 epoch  :  1678.5081391334534 [sec] → 2021-09-26 21:38:11.195667+09:00

5 epoch
5 epoch  :  2013.988648891449 [sec] → 2021-09-26 21:43:46.676186+09:00

6 epoch
6 epoch  :  2349.604227781296 [sec] → 2021-09-26 21:49:22.291751+09:00

7 epoch
7 epoch  :  2684.9856145381927 [sec] → 2021-09-26 21:54:57.673144+09:00

8 epoch
8 epoch  :  3020.727757692337 [sec] → 2021-09-26 22:00:33.415285+09:00

9 epoch
9 epoch  :  3356.3819003105164 [sec] → 2021-09-26 22:06:09.069429+09:00

終了時間 : 3356.3819003105164 [sec] → 2021-09-26 22:06:09.069938+09:00


In [ ]:
train_loss_

[52.3909749686718,
 50.0940320789814,
 42.17575464397669,
 26.88246811926365,
 15.657660521566868,
 10.478117975406349,
 7.355932934675366,
 6.749802705598995,
 5.0598999571520835,
 4.832028003875166]

In [ ]:
test_loss_

[11.079733550548553,
 11.332718014717102,
 12.256916493177414,
 16.269230037927628,
 19.53595933318138,
 24.711122512817383,
 25.490011036396027,
 26.250460624694824,
 30.242721736431122,
 31.27320808172226]

In [ ]:
 test_

31.27320808172226

#6. 検証

In [ ]:
# 検証方法の確認（1バッチ分で計算ロジックに確認）

model.eval()# 訓練モードをオフ
for batch in validation_dataloader:
    b_input_ids = batch[0].to(device)
    b_input_mask = batch[1].to(device)
    b_labels = batch[2].to(device)
    with torch.no_grad():   
        # 学習済みモデルによる予測結果をpredsで取得     
        preds = model(b_input_ids, 
                            token_type_ids=None, 
                            attention_mask=b_input_mask)

以下の様な結果が出力されます
この確率の様な値（厳密には、この値ををソフトマックス関数に入力すると確率になる）が大きい方のラベルをモデルは分類結果とし予測としています

In [ ]:
## 予測結果の確認
print(f'出力:{preds}')

出力:SequenceClassifierOutput(loss=None, logits=tensor([[ 4.3053, -3.8184],
        [ 3.9481, -3.8118],
        [ 3.7653, -3.4892],
        [ 3.6510, -3.6121],
        [ 3.9619, -3.6851],
        [ 3.8946, -3.7065],
        [ 3.6635, -3.5523],
        [ 3.8446, -3.8363],
        [ 3.8124, -3.5514],
        [ 4.1857, -4.0212],
        [ 4.3325, -4.1140],
        [ 4.4919, -4.2100],
        [ 2.8937, -2.7336],
        [ 4.4674, -4.1326],
        [ 2.9164, -2.8657],
        [ 3.9297, -3.6368],
        [ 4.6054, -4.1863],
        [ 4.0404, -3.6373],
        [ 4.0287, -3.8354],
        [-1.0860,  0.9149],
        [ 1.3303, -1.1790],
        [ 3.9542, -3.8564],
        [ 4.0305, -3.8604],
        [ 3.6734, -3.5879],
        [ 3.7138, -3.7110],
        [ 2.8915, -2.8946],
        [ 2.1294, -1.8539],
        [ 2.5663, -2.5085],
        [ 3.2512, -3.2969],
        [ 2.4247, -2.4408],
        [ 3.9324, -3.9095],
        [ 1.9324, -1.9749],
        [ 4.1568, -3.7743],
        [ 3.5571, -3.6793],
  



```
出力:(tensor([[-5.0226,  5.0193],
        [-5.0390,  4.9736],
        [ 4.7941, -4.7459],
        [-4.9395,  4.6827], device='cuda:0'),)
```



左からラベルの[0, 1]に対応しており、1つ目のデータであれば、右側の値が大きいのでモデルはラベル1を予測している

In [ ]:
# 比較しやすい様にpd.dataframeへ整形
import numpy as np
import pandas as pd
# pd.dataframeへ変換（GPUに乗っているTensorはgpu->cpu->numpy->dataframeと変換）
logits_df = pd.DataFrame(preds[0].cpu().numpy(), columns=['logit_0', 'logit_1'])
## np.argmaxで大き方の値を取得
pred_df = pd.DataFrame(np.argmax(preds[0].cpu().numpy(), axis=1), columns=['pred_label'])
label_df = pd.DataFrame(b_labels.cpu().numpy(), columns=['true_label'])

accuracy_df = pd.concat([logits_df, pred_df, label_df], axis=1)

accuracy_df.head()

accuracy_df.to_csv('./%s.csv' % TYPE_INDEX)

In [ ]:
accuracy_df

,logit_0,logit_1,pred_label,true_label
0,4.305343,-3.818410,0,0
1,3.948097,-3.811775,0,0
2,3.765281,-3.489188,0,0
3,3.651007,-3.612106,0,0
4,3.961890,-3.685114,0,0
...,...,...,...,...
59,4.494333,-4.146439,0,0
60,4.438259,-3.989540,0,0
61,3.033069,-2.704979,0,0
62,4.210387,-4.048031,0,0


In [ ]:
accuracy_df['answer'] = (accuracy_df['pred_label'] == accuracy_df['true_label'])
list_answer=accuracy_df['answer'].to_list()
count_true=list_answer.count(True)
print(list_answer)
print(count_true)
print('accuracy : ',  float(count_true / len(list_answer))*100)

[True, True, True, True, True, True, True, True, True, True, True, True, False, True, True, True, True, True, True, False, True, False, True, True, True, True, True, True, False, True, True, True, True, True, False, True, False, True, True, True, True, True, True, False, True, False, True, False, True, True, True, True, False, False, True, True, True, True, True, True, True, True, True, False]
52
accuracy :  81.25


In [ ]:
pred_df.values

array([[0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [1],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [1],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0]])

上記の様に予測ラベルと正解ラベルを取得して、dataframeへ保存することができました

あとは、シンプルに正解率を計算するなり、sklearnを用いて混合行列を作成するなり、好きな様に分類モデルの評価を行うことができる。